Below is boilerplate items for each of these analysis notebooks.

In [1]:
DATA_FOLDER = 'data/so/'

In [2]:
from IPython.display import display, HTML
import numpy as np

Import my own custom handlers

In [3]:
import altair as alt
alt.renderers.enable('notebook')

from viz import visualize_seqs
from model_stats import ModelStats

results = ModelStats()

# Data - StackOverflow

The data for StackOverflow is based off of the badges that users received.

Read data

In [4]:
file_index = 1
events = {}
times = {}

for data_type in ['train', 'test']:
    with open(DATA_FOLDER + 'event-' + str(file_index) + '-' + data_type + '.txt') as f_events:
        events[data_type] = [[int(y) for y in x.split()] for x in f_events]
        
    with open(DATA_FOLDER + 'time-' + str(file_index) + '-' + data_type + '.txt') as f_times:
        times[data_type] = [[float(y) for y in x.split()] for x in f_times]

Compute deltas for times.

It seems that the general approach usually is to just use timestamps, but going to also compute deltas because maybe will want to use that to make things easier.

In [5]:
deltas = {}
for data_type in ['train', 'test']:
    deltas[data_type] = []
    for seq in times[data_type]:
        seq_deltas = []
        for i in range(1, len(seq)):
            seq_deltas.append(seq[i] - seq[i-1])
        deltas[data_type].append(seq_deltas)

# Models for Both

Errors for these models is RMSE:

https://en.wikipedia.org/wiki/Root-mean-square_deviation

# Helper Functions

In [6]:
def make_flat(arr):
    flattened = []
    for seq in arr:
        flattened += seq
        
    return flattened

In [7]:
def convert_flat(predictions, data_type):
    pred = []
    index = 0
    for s, seq in enumerate(events[data_type]):
        start_index = index
        index += len(seq) - 1
        pred.append(predictions[start_index:index])
        
    return pred

In [8]:
def convert_nn_seq(predictions, data_type):
    pred = []
    for s, seq in enumerate(events[data_type]):
        seq_len = len(seq) - 1
        pred.append(predictions[s][-seq_len:].tolist())
        
    return pred

In [13]:
from sklearn.metrics import mean_squared_error
import math

def compute_event_error(y_true, y_pred):
    num_error = 0
    total = 0
    for s, seq in enumerate(y_true):
        total += len(seq) - 1
        for e, event in enumerate(seq[1:]):
            if event != y_pred[s][e]:
                num_error += 1
                
    return (num_error / total) * 100

def compute_time_error(y_true, y_pred):
    y_true = make_flat(y_true)
    y_pred = make_flat(y_pred)
    
    return math.sqrt(mean_squared_error(y_true, y_pred))

def show_results(name, train_pred, test_pred):
    print('----')
    train_event_err = compute_event_error(events['train'], train_pred[0])
    train_time_err = compute_time_error(deltas['train'], train_pred[1])
    test_event_err = compute_event_error(events['test'], test_pred[0])
    test_time_err = compute_time_error(deltas['test'], test_pred[1])
    results.add_stat(name, {'train': train_event_err, 'test': test_event_err}, {'train': train_time_err, 'test': test_time_err})
    print('Train Error: ', train_event_err, '-', train_time_err)
    print('Test Error: ', test_event_err, '-', test_time_err)
    print(len(events['train']))
    print(len(events['train'][0]))
    print(len(train_pred[0]))
    print(len(train_pred[0][0]))
    print('--')
    print(len(deltas['train']))
    print(len(deltas['train'][0]))
    print(len(train_pred[1]))
    print(len(train_pred[1][0]))
    display(HTML('<h3>Train</h3>'))
    display(HTML(visualize_seqs(events['train'], train_pred[0], deltas['train'], train_pred[1])))
    display(HTML('<h3>Test</h3>'))
    display(HTML(visualize_seqs(events['test'], train_pred[0], deltas['test'], test_pred[1])))

In [10]:
def show_results_nn(name, train_preds, test_preds):
    # Convert event predictions
    train_preds[0] = np.argmax(train_preds[0], axis=2) + 1
    test_preds[0] = np.argmax(test_preds[0], axis=2) + 1
    
    train_preds[1] = np.squeeze(train_preds[1], axis=2)
    test_preds[1] = np.squeeze(test_preds[1], axis=2)
    
    for i in range(2):
        train_preds[i] = convert_nn_seq(train_preds[i], 'train')
        test_preds[i] = convert_nn_seq(test_preds[i], 'test')
        
    show_results(name, train_preds, test_preds)

# Baseline Predictor

Using methodology from https://github.com/dunan/NeuralPointProcess/blob/master/code/baselines/majority_predictor/time_mean_baseline.py

A scale is used there so adding a variable for that.

In [11]:
scale = 0.001

In [14]:
# Predict events
from collections import Counter

cnt = Counter()
for seq in events['train']:
    for event in seq:
        cnt[event] += 1

top = cnt.most_common(1)[0][0]
print('Most commom event is', top)

train_pred_events = [[top for event in seq[1:]] for seq in events['train']]
test_pred_events = [[top for event in seq[1:]] for seq in events['test']]


# Predict deltas
train_deltas = make_flat(deltas['train'])
train_deltas = np.asarray(train_deltas) * scale

mean_delta = np.mean(train_deltas)
print('Mean predictor using deltas', mean_delta)

train_pred_deltas = [[mean_delta for d in seq] for seq in deltas['train']]
test_pred_deltas = [[mean_delta for d in seq] for seq in deltas['test']]


# Show results
show_results('Majority', (train_pred_events, train_pred_deltas), (test_pred_events, test_pred_deltas))

Most commom event is 4
Mean predictor using deltas 828.9236502234185
----
Train Error:  56.53630575271122 - 1332529.9103083226
Test Error:  57.106363456264916 - 1321285.6966187225
5307
44
5307
43
--
5307
43
5307
43


IndexError: list index out of range

# Structure Data

Need to structure the data into entire sequences for the approaches

Normalize deltas for the neural network approaches.  Could try alternate normalization techniques in the future.

In [ ]:
# Get the min and max deltas
min_delta = 999999
max_delta = -1

for data_type in ['train', 'test']:
    for seq in deltas[data_type]:
        if min(seq) < min_delta:
            min_delta = min(seq)
            
        if max(seq) > max_delta:
            max_delta = max(seq)
            

print('Min delta', min_delta)
print('Max delta', max_delta)

deltas_temp = {}
# Normalize based on max, just going to ignore min and treat it like it was 0
for data_type in ['train', 'test']:
    deltas_temp[data_type] = [[d / max_delta for d in seq] for seq in deltas[data_type]]
    
# # Function to undo normalize
# def undo_norm_deltas(preds):
#     for data_type in ['train', 'test']:
    

In [ ]:
x = {}
y = {}

for data_type in ['train', 'test']:
    x[data_type] = [[], []]
    y[data_type] = [[], []]
    
    # Events
    for seq in events[data_type]:
        x[data_type][0].append(seq[:-1])
        y[data_type][0].append(seq[1:])
        
    # Deltas
    for seq in deltas_temp[data_type]:
        x[data_type][1].append(seq[:-1])
        y[data_type][1].append(seq[1:])

    for i in range(2):
        x[data_type][i] = np.asarray(x[data_type][i])
        y[data_type][i] = np.asarray(y[data_type][i])

In [ ]:
# Find the max user length
user_lengths = [len(seq) for seq in events['train']] + [len(seq) for seq in events['test']]
user_length_counts = {}
for length in user_lengths:
    if length not in user_length_counts:
        user_length_counts[length] = 0
        
    user_length_counts[length] += 1
user_lengths = list(set(user_lengths))
user_lengths = sorted(user_lengths, reverse=True)
max_user_length = max(user_lengths)

print('Max Seq Length', max_user_length)
# print(user_lengths)

Subtract one from x for events

In [ ]:
for data_type in ['train', 'test']:
    x[data_type][0] = [[event - 1 for event in seq] for seq in x[data_type][0]]

In [ ]:
# Pad sequences
from keras.preprocessing import sequence

for data_type in ['train', 'test']:
    for i in range(2):
        x[data_type][i] = sequence.pad_sequences(x[data_type][i], maxlen=max_user_length)
        y[data_type][i] = sequence.pad_sequences(y[data_type][i], maxlen=max_user_length)

y variable needs to be one-hot for events.  (x will be made one-hot later for different models)

In [ ]:
num_events = 22
one_hot = np.eye(num_events)
for data_type in ['train', 'test']:
    y[data_type][0] = [[one_hot[event-1] if event > 0 else np.zeros(num_events) for event in seq] for seq in y[data_type][0]]
    y[data_type][0] = np.asarray(y[data_type][0])

Add dimension to the end

In [ ]:
x['train'][0] = np.expand_dims(x['train'][0], axis=2)
x['train'][1] = np.expand_dims(x['train'][1], axis=2)
y['train'][1] = np.expand_dims(y['train'][1], axis=2)

x['test'][0] = np.expand_dims(x['test'][0], axis=2)
x['test'][1] = np.expand_dims(x['test'][1], axis=2)
y['test'][1] = np.expand_dims(y['test'][1], axis=2)

# Neural Networks

We are going to try a bunch of different network configurations so let's import stuff that we'll need.

In [ ]:
from keras.models import Sequential, Model
from keras import optimizers
from keras.layers import Input, Dense, TimeDistributed, Masking, SimpleRNN, Concatenate, RNN, LSTM

# TODO - Evaluate how categorical_accuracy works for metrics

In [ ]:
input_events = Input(shape=(max_user_length, 1))
input_deltas = Input(shape=(max_user_length, 1))

inputs = Concatenate(axis=2)([input_events, input_deltas])
inputs = Masking(mask_value=0.)(inputs)

rnn = SimpleRNN(50, activation='tanh', return_sequences=True)(inputs)

output_events = TimeDistributed(Dense(num_events, activation='softmax'), name='events')(rnn)

output_deltas = TimeDistributed(Dense(15, activation='relu'))(rnn)
output_deltas = TimeDistributed(Dense(1, activation='sigmoid'), name='time')(output_deltas)

model = Model(inputs=[input_events, input_deltas], outputs=[output_events, output_deltas])

model.compile(loss=['categorical_crossentropy', 'mse'], optimizer='adam', metrics={'events': 'accuracy'})

model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']), batch_size=100, epochs=3)

preds_train = model.predict(x['train'])
preds_test = model.predict(x['test'])

preds_train[1] *= max_delta
preds_test[1] *= max_delta

show_results_nn('RNN', preds_train, preds_test)

In [ ]:
input_events = Input(shape=(max_user_length, 1))
input_deltas = Input(shape=(max_user_length, 1))

inputs = Concatenate(axis=2)([input_events, input_deltas])
inputs = Masking(mask_value=0.)(inputs)

rnn = LSTM(50, activation='tanh', return_sequences=True)(inputs)

output_events = TimeDistributed(Dense(num_events, activation='softmax'), name='events')(rnn)

output_deltas = TimeDistributed(Dense(15, activation='relu'))(rnn)
output_deltas = TimeDistributed(Dense(1, activation='sigmoid'), name='time')(output_deltas)

model = Model(inputs=[input_events, input_deltas], outputs=[output_events, output_deltas])

model.compile(loss=['categorical_crossentropy', 'mse'], optimizer='adam', metrics={'events': 'accuracy'})

model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']), batch_size=100, epochs=3)

preds_train = model.predict(x['train'])
preds_test = model.predict(x['test'])

preds_train[1] *= max_delta
preds_test[1] *= max_delta

show_results_nn('LSTM', preds_train, preds_test)

## Phased-LSTM Model

In [ ]:
from phasedlstmcell import PhasedLSTMCell

input_events = Input(shape=(max_user_length, 1))
input_deltas = Input(shape=(max_user_length, 1))

inputs = Concatenate(axis=2)([input_events, input_deltas])
inputs = Masking(mask_value=0.)(inputs)

cell = PhasedLSTMCell(50, activation='tanh')
rnn = RNN(cell, return_sequences=True)(inputs)

output_events = TimeDistributed(Dense(num_events, activation='softmax'), name='events')(rnn)

output_deltas = TimeDistributed(Dense(15, activation='relu'))(rnn)
output_deltas = TimeDistributed(Dense(1, activation='sigmoid'), name='time')(output_deltas)

model = Model(inputs=[input_events, input_deltas], outputs=[output_events, output_deltas])

model.compile(loss=['categorical_crossentropy', 'mse'], optimizer='adam', metrics={'events': 'accuracy'})

model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']), batch_size=100, epochs=3)

preds_train = model.predict(x['train'])
preds_test = model.predict(x['test'])

preds_train[1] *= max_delta
preds_test[1] *= max_delta

show_results_nn('PhasedLSTM', preds_train, preds_test)

# Model Stats

## Event Error

In [ ]:
results.grouped_bar('event')

### Train

In [ ]:
results.graph_results('event', 'train')

### Test

In [ ]:
results.graph_results('event', 'test')

## Time Error

In [ ]:
results.grouped_bar('time')

### Train

In [ ]:
results.graph_results('time', 'train')

### Test

In [ ]:
results.graph_results('time', 'test')